# NARX


In [1]:
# @title Lendo a planilha

import numpy as np
import pandas as pd

df = pd.read_csv("../export/dados.csv", index_col=0)

ciclos = df.index.values
t_feed = df["t_feed"].values
t_rinse = df["t_rinse"].values
t_blow = df["t_blow"].values
t_purge = df["t_purge"].values

purity_H2 = df["purity_H2"].values
H2_CO_ratio = df["H2_CO_ratio"].values
purity_CO2 = df["purity_CO2"].values
recovery_CO2 = df["recovery_CO2"].values
productivity = df["productivity"].values


model_name = "NARX"


In [2]:
# Colocando dados em desvio

y = np.array(
    [
        purity_H2,
        H2_CO_ratio,
        purity_CO2,
        recovery_CO2,
        productivity,
    ]
).T
u = np.array([t_feed, t_rinse, t_blow, t_purge]).T

# ciclo de referência
ref_idx = 1
y_ref = np.tile(y[ref_idx, :], (len(y), 1))
u_ref = np.tile(u[ref_idx, :], (len(u), 1))

# --- Calcular desvios ---
y_desvio = y - y_ref
u_desvio = u - u_ref


In [3]:
from lib.utils import benchmark


def gerar_NARX(y, u):
    y_prev = y[:-1, :]
    u_prev = u[:-1, :]
    u_now = u[1:, :]

    # Termos lineares
    X_lin = np.hstack([y_prev, u_prev, u_now])
    # Termos quadráticos (não-lineares)
    X_quad = np.hstack([y_prev**2, u_prev**2, u_now**2])
    # Termos de interação
    X_inter = np.hstack(
        [
            (y_prev[:, :, None] * u_prev[:, None, :]).reshape(len(y_prev), -1),
            (y_prev[:, :, None] * u_now[:, None, :]).reshape(len(y_prev), -1),
            u_prev * u_now,
        ]
    )

    # Matriz de regressão final
    X = np.hstack([X_lin, X_quad, X_inter])

    # Saídas futuras
    Y = y[1:, :]

    # Estimando parâmetros via mínimos quadrados
    theta = np.linalg.lstsq(X, Y)[0]
    return theta


def inferir_NARX(theta, y, u):
    N = len(y)
    y_narx = np.zeros_like(y)
    y_narx[0] = y[0]  # inicializa com o primeiro valor real

    for k in range(1, N):
        yk = y_narx[k - 1]
        uk_prev = u[k - 1]
        uk_now = u[k]

        # lineares
        phi_lin = np.concatenate([yk, uk_prev, uk_now])

        # quadráticos
        phi_quad = np.concatenate([yk**2, uk_prev**2, uk_now**2])

        # interações
        phi_inter = np.concatenate(
            [
                (yk[:, None] * uk_prev[None, :]).reshape(-1),
                (yk[:, None] * uk_now[None, :]).reshape(-1),
                (uk_prev * uk_now),
            ]
        )

        phi = np.concatenate([phi_lin, phi_quad, phi_inter])

        y_narx[k] = phi @ theta
    return y_narx


theta = gerar_NARX(y_desvio, u_desvio)
print("Parâmetros estimados:\n", theta)

y_narx = inferir_NARX(theta, y_desvio, u_desvio)
y_narx = y_narx + y_ref  # Convertendo o valor desvio em variavel de engenharia

# Benchmarking
benchmark(inferir_NARX, model_name, theta, y_desvio, u_desvio)


Parâmetros estimados:
 [[ 4.42135431e-01  8.80195246e-03  4.42712027e-01  2.06003392e+00
   8.80955687e-02]
 [-1.64002489e+01  1.21050351e+00  3.80527668e+01  3.33411666e+01
   1.22449893e+00]
 [ 3.66162961e-01 -1.77685818e-02 -5.02732216e-01 -1.21570634e-01
  -2.75723930e-03]
 [-2.29673323e-01  2.37186418e-02  8.70589645e-01  1.06347767e+00
   4.36626662e-02]
 [ 3.32116524e+00 -3.33703059e-01 -1.15302091e+01 -1.18243308e+01
  -4.78609593e-01]
 [-6.84238218e-04  1.68768397e-03  3.80239474e-02  1.83498536e-02
   1.78275954e-03]
 [-4.50065679e-02  2.32352011e-03  1.15199636e-01  1.13494381e-01
   3.34389754e-03]
 [ 1.08222102e-02 -1.54695834e-03 -3.55395688e-02 -1.90894670e-02
  -2.16737481e-03]
 [ 6.84076691e-02 -3.97191188e-03 -2.13264007e-01 -9.24351652e-02
  -4.76632846e-03]
 [-3.31834944e-02  3.15970184e-04  5.35289617e-02  3.89587540e-02
   3.76016322e-03]
 [-1.18450816e-02 -1.36547171e-03  1.99332647e-02 -1.15749188e-01
  -7.53593655e-03]
 [ 2.93681536e-02 -4.69822924e-04 -4.19723

In [4]:
# Plotando resultados
from lib.plot import plot_comparison

plot_comparison(ciclos, y, y_narx, model_name)
